# Demo Config

> Demo of a congiguration file with database paths, urls etc...

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# | default_exp demo_config

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | export
from dataclasses import dataclass, field
from hydra.core.config_store import ConfigStore
from typing import Any
from omegaconf import MISSING
from hydra.conf import HydraConf, RunDir

In [ ]:
# | exports
# | code-fold: true

@dataclass(frozen=True)
class MySQLConfig:
    host: str = "localhost"
    port: int = 3306

@dataclass
class UserInterface:
    title: str = MISSING
    width: int = MISSING
    height: int = MISSING

@dataclass
class UserInterface1(UserInterface):
    title: str = "UI1"
    width: int = 1024
    height: int = 768
    
@dataclass
class UserInterface2(UserInterface):
    title: str = "My other app"
    width: int = 1024
    height: int = 768
    new: str = "ui1 doens't have this attr"

@dataclass
class CustomHydraConf(HydraConf):
    """For now stick with dataclass"""
    run: RunDir = field(default_factory=lambda: RunDir("../yewster/${now:%Y-%m-%d}/${now:%H-%M-%S}"))
    # sweep: SweepDir = field(default_factory=lambda: SweepDir(dir="custom_name/${now:%Y-%m-%d}/${now:%H-%M-%S}", subdir="${hydra.job.num}"))



# note that overrides must be at end of list. note that including "_self_" as the last
# entry before the overrides does not change the behavior, but simply suppresses the
# warning. It dictates the order in which the config is merged, here it's command line args
# take precedence over defaults. 
defaults = [{"ui": "ui1"}, "_self_", {"override hydra/launcher": "joblib"}]
@dataclass
class MainConfig:
    defaults: list[Any] = field(default_factory=lambda: defaults)
    hydra: CustomHydraConf = field(default_factory=CustomHydraConf)
    db: MySQLConfig = field(default_factory=MySQLConfig)
    ui: UserInterface = MISSING
    
    # the below doesn't run for calling ui=ui2 as UserInterface1 is not a subclass of UserInterface2
    # ui: UserInterface = field(default_factory=UserInterface1)
    # its just unfortunate that the defaults are kinda split because of this
    # some in the defaults list (like ui as the classes have different structures) and some
    # just having the default factory (like db)

def register_configs() -> None:
    cs = ConfigStore.instance()
    cs.store(group="ui", name="ui1", node=UserInterface1)
    cs.store(group="ui", name="ui2", node=UserInterface2)
    cs.store(name="config", node=MainConfig)

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()